# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display
import toml
import gspread
from google.oauth2.service_account import Credentials
from auxiliar.arrumar_bases import *

def get_sheetdata(sheet_name):
    # Load credentials from the .streamlit/secrets.toml file
    secrets = toml.load(".streamlit/secrets.toml")
    creds_info = secrets["connections"]["gsheets"]

    # Define scopes for accessing Google Sheets and Drive
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    # Create credentials and authorize the gspread client
    creds = Credentials.from_service_account_info(creds_info, scopes=scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet using the URL from the toml file
    spreadsheet = client.open_by_url(creds_info["spreadsheet"])

    # Select the first worksheet and fetch all records
    worksheet = spreadsheet.worksheet(sheet_name)
    data = worksheet.get_all_records()

    df = pd.DataFrame(data)

    return df
    
vmb_path = "assets/venda-mensal-bruta_colab-bi_2025-02-11.xlsx"
agendamento_path = "assets/agendamentos_colab-bi_2025-02-12 (1).xlsx"

venda_mensal_df = pd.read_excel(vmb_path)
agendamentos_df = pd.read_excel(agendamento_path)
procedimentos_df = get_sheetdata("procedimentos_padronizados")
prestadora_df = get_sheetdata("base_prestadoras")
comissao_df = get_sheetdata("comissoes")
tipo_prestadora_df = get_sheetdata("tipo_prestadora")



# Visualizar

In [2]:
base_compilada = criar_base_compilada(agendamentos_df,venda_mensal_df,procedimentos_df,prestadora_df,comissao_df,tipo_prestadora_df)
periodos = base_compilada["periodo"].unique()
periodos = pd.Series(periodos)
s_date_str = "periodo: " + periodos.dt.strftime('%Y-%m-%d')


base_compilada.columns

base_comissoes = criar_comissoes(base_compilada)

base_avaliacoes = cria_avaliacoes(base_compilada,"mensal")
base_revenda = cria_base_revenda(venda_mensal_df)

base_final = juntar_bases(base_comissoes,base_avaliacoes,base_revenda)

In [5]:
base_revenda

,Unidade,nome_prestadora,comissao_revenda
0,GUARULHOS,AMANDA SOUZA FAIOLI,172.91757
1,TUCURUVI,AMANDA SOUZA FAIOLI,477.08243
2,LAPA,ANA LUCIA DE JESUS DENIS,457.47711
3,SANTO AMARO,ANA LUCIA DE JESUS DENIS,1142.52289
4,CAMPINAS,ANA PAULA DE ALENCAR BENASSI ROCHA,300.00000
...,...,...,...
149,JARDINS,Tatiana Cristina Bartolomeu Angelo Ricardo,0.00000
150,COPACABANA,Thais Ferreira Rodrigues dos Santos,0.00000
151,PRAIA GRANDE,Thalita Ferreira Domingues,0.00000
152,TIJUCA,VIVIAN FALHEIRO DE LIMA,0.00000


In [4]:
base_final.sample(50)

,nome_prestadora,tipo_prestadora,Unidade,comissao_total,avaliacoes_total,comissao_revenda
52,laís ferreira da silva,prestação de serviços - biomedica,osasco,7414.0,600.0,NaN
25,cintia serafim,prestação de serviços - dentista,moema,3079.0,50.0,NaN
66,milena riesco de jesus pereira,prestação de serviços - biomedica,santos,2107.0,NaN,NaN
4,amanda cristina soares da silva,salão parceiro,tijuca,10645.0,NaN,NaN
49,júlia dos santos tavares,prestação de serviços - biomedica,tatuapé,1658.0,20.0,NaN
51,karine gomes dos santos,salão parceiro,tucuruvi,6150.0,140.0,NaN
85,simone crocci ferreira,personal estetic clt,santo amaro,0.0,NaN,NaN
82,sabrina barbosa juvencio,prestação de serviços - biomedica,tatuapé,1434.0,60.0,NaN
73,patrícia monique groba smith howard,prestação de serviços - biomedica,sorocaba,18.0,NaN,NaN
12,andressa de mello guimarães,salão parceiro,tijuca,7908.0,NaN,NaN
